In [0]:
 filename = dbutils.widgets.get('filename') # The filename will be taken from the pipeline
 fnamewithoutExt = filename.split('.')[0]
 print(filename)

##setting generic mountcode


In [0]:
alreadymounted=False
for x in dbutils.fs.mounts():
    if x.mountPoint == '/mnt/sales':
        alreadymounted=True
        break
    else:
        alreadymounted=False

print(alreadymounted)

In [0]:
 dbServer='analysisserver'
 dbPort=1433
 dbName='salesdb'
 dbUser='yashwanth'
 dbPassword='sql-password'
 databricksScope='databricks-scope'

In [0]:
storagekey=dbutils.secrets.get(scope='databricks-scope',key='storageaccount-key')

In [0]:
if not alreadymounted:
     dbutils.fs.mount(
     source='wasbs://sales@sttacntsales.blob.core.windows.net',
     mount_point = '/mnt/sales',
     extra_configs={'fs.azure.account.key.sttacntsales.blob.core.windows.net':storagekey})
     alreadymounted=True
     print("mounting done sucessfully")
else:
    print(" its already mounted")


In [0]:
 
ordersdf=spark.read.csv('/mnt/sales/landing/{}'.format(filename),header=True,inferSchema=True)
display(ordersdf)


##1stcondition

In [0]:
errorflag=False

orderscount=ordersdf.count()

ordersdistinctcount=ordersdf.select('order_id').distinct().count()

if orderscount!=ordersdistinctcount:
    errorflag=True

if errorflag:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit({"errorflag":"flag","errormsg":"orderid is repeated"})

ordersdf.createOrReplaceTempView("orders")




In [0]:
connectionUrl ='jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbPort, dbName, dbUser)

dbPassword = dbutils.secrets.get(scope=databricksScope,
 key='sql-password')

connectionProperties = {
 'password' : dbPassword,
 'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
 }





In [0]:
 validStatusDf = spark.read.jdbc(url =connectionUrl, table =
 'dbo.valid_order_status', properties = connectionProperties)

In [0]:
validStatusDf.createOrReplaceTempView("validorders")

invalidorders=spark.sql("select * from orders where order_status not in(select * from validorders)")
 
display(invalidorders)


##2ndcondition

In [0]:
if invalidorders.count() > 1:
    errorflag=True
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit({"errorflag":"true","errormsg":"invalid order status"})
else:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/staging')
    
    



##validations


In [0]:
customers=spark.read.jdbc(url=connectionUrl,table='dbo.customers',properties=connectionProperties)
customers.createOrReplaceTempView("customers")
orderitems=spark.read.csv("/mnt/sales/orderitems/order_items.csv",header=True,inferSchema=True)
orderitems.createOrReplaceTempView("orderitems")


In [0]:
order_df=spark.read.csv("/mnt/sales/staging/{}".format(filename),header=True,inferSchema=True)
order_df.createOrReplaceTempView("orders_1")

In [0]:
result_df=spark.sql("""select c.customer_id,c.customer_fname,c.customer_lname,count(o.order_id),sum(o1.order_item_subtotal) from customers as c inner join orders_1 as o on c.customer_id=o.customer_id inner join orderitems o1 on o1.order_item_order_id=o.order_id group by c.customer_id,c.customer_fname,c.customer_lname""")

result_df.write.jdbc(url=connectionUrl.table='dbo.reportingsales',mode='overwrite',properties=connectionProperties)

{"ts": "2025-11-03 20:48:32.851", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `orders` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o410.sql.\n: org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `orders` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5584729881864950>, line 1
----> 1 spark.sql("""select c.customer_id,c.customer_fname,c.customer_lname,count(o.order_id),sum(o1.order_item_subtotal) from customers as c inner join orders as o on c.customer_id=o.customer_id inner join orderitems o1 on o1.order_item_order_id=o.order_id group by c.customer_id,c.customer_fname,c.customer_lname""")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1868, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1